In [9]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from gensim.models import word2vec

In [10]:
model=word2vec.Word2Vec.load("/Users/lwd011204/书籍爬虫/book_spyder/resources/word2vec.model")
model.vector_size,len(model.wv)

(100, 17174)

In [11]:
class config:
    num_class=15
    dropout_rate=0.1
    max_text_length=46
    vocab_size=17146
    vec_dim=100
    kernel_size=[2,3,4]
    feature_size=100#一维卷积卷积核个数
    weight=torch.tensor(model.wv.vectors)
    vocab_size=len(model.wv)


In [12]:
config.weight

tensor([[-3.4236e-03,  7.6554e-04,  6.5606e-03,  ..., -1.0835e-02,
          7.6421e-05,  9.6728e-03],
        [-9.4926e-03,  6.1837e-03,  5.6921e-03,  ..., -4.5948e-03,
         -9.0215e-03,  5.7719e-03],
        [ 2.9097e-04,  4.6717e-03, -6.3387e-03,  ..., -7.9938e-04,
          8.8260e-03, -6.2926e-03],
        ...,
        [-8.7676e-03,  4.4933e-03, -3.3572e-03,  ...,  3.6418e-03,
         -1.1014e-03, -5.5919e-04],
        [-5.8528e-03,  2.6498e-04,  8.7569e-03,  ...,  5.2154e-03,
         -7.4856e-03, -3.7171e-03],
        [-4.8476e-03,  8.8799e-03,  1.4623e-03,  ..., -8.6098e-03,
         -5.6974e-04, -5.4949e-03]])

In [13]:
class TextCNN(nn.Module):
    def __init__(self,config) -> None:
        super(TextCNN,self).__init__()
        self.dropout_rate=config.dropout_rate
        self.num_class=config.num_class
        self.config=config
        self.embedding=nn.Embedding(config.vocab_size,config.vec_dim)
        self.convs=nn.ModuleList([nn.Sequential(nn.Conv1d(in_channels=config.vec_dim,out_channels=config.feature_size,kernel_size=h),
        nn.ReLU(),nn.MaxPool1d(kernel_size=config.max_text_length-h+1)) 
        for h in config.kernel_size])
        self.fc=nn.Linear(config.feature_size*len(config.kernel_size),out_features=config.num_class)
        self.init_weight()
    def init_weight(self,config):
        self.embedding.weight.data.copy(config.weight)
    def forward(self,x):
        embed_x=self.embedding(x)
        embed_x=embed_x.permute(0,2,1)
        out=[conv(embed_x) for conv in self.convs]
        out=torch.cat(out,dim=1)
        out=out.view(-1,out.size(1))
        out=F.dropout(out,p=self.dropout_rate)
        out=self.fc(out)
        return out

In [14]:
net=TextCNN(config)

AttributeError: type object 'config' has no attribute 'kernel_sizes'